# Обучение и сравнение моделей машинного обучения

На данном этапе производится обучение и сравнение различных алгоритмов машинного обучения для классификации писем на легитимные и фишинговые. Для каждого алгоритма выполняется подбор гиперпараметров, кросс-валидация и оценка на валидационной выборке. Затем выбирается лучшая модель, выполняется калибровка весов агрегации эвристического модуля и ML-модели, и финальная оценка на тестовой выборке.


## 1. Загрузка данных

На данном этапе производится загрузка предобработанных данных из директории `/data/models/`:

- `train.pkl` — обучающая выборка (X_train, y_train)
- `val.pkl` — валидационная выборка (X_val, y_val)  
- `test.pkl` — тестовая выборка (X_test, y_test)
- `tfidf_vectorizer.pkl` — обученный векторизатор для проверки консистентности

Данные были предобработаны и сохранены в блокноте `dataset_and_features.ipynb`.


In [ ]:
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import time
import json
import warnings
warnings.filterwarnings('ignore')
import sys

# Импорт библиотек для машинного обучения
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_validate, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)
from scipy.stats import loguniform, randint, uniform
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Определение путей
BASE_DIR = Path('../').resolve()
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))

# Настройка визуализации
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Определение путей к данным
DATA_DIR = BASE_DIR / 'data' / 'models'
MODELS_DIR = BASE_DIR / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Загрузка данных
print("Загрузка train.pkl...")
with open(DATA_DIR / 'train.pkl', 'rb') as f:
    train_data = pickle.load(f)
    X_train = train_data['X_train']
    y_train = train_data['y_train']

print("Загрузка val.pkl...")
with open(DATA_DIR / 'val.pkl', 'rb') as f:
    val_data = pickle.load(f)
    X_val = val_data['X_val']
    y_val = val_data['y_val']

print("Загрузка test.pkl...")
with open(DATA_DIR / 'test.pkl', 'rb') as f:
    test_data = pickle.load(f)
    X_test = test_data['X_test']
    y_test = test_data['y_test']

print("Загрузка tfidf_vectorizer.pkl...")
with open(DATA_DIR / 'tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer_data = pickle.load(f)
    tfidf_vectorizer = vectorizer_data['vectorizer']

print("\n✓ Все данные загружены успешно!")
print(f"\nРазмерности данных:")
print(f"  Train:      {X_train.shape[0]} образцов, {X_train.shape[1]} признаков")
print(f"  Validation: {X_val.shape[0]} образцов, {X_val.shape[1]} признаков")
print(f"  Test:       {X_test.shape[0]} образцов, {X_test.shape[1]} признаков")
print(f"\nРаспределение классов в train:")
print(f"  Legitimate (0): {np.sum(y_train == 0)} ({np.sum(y_train == 0)/len(y_train)*100:.1f}%)")
print(f"  Phishing (1):   {np.sum(y_train == 1)} ({np.sum(y_train == 1)/len(y_train)*100:.1f}%)")


## 2. Подбор гиперпараметров и обучение моделей

На данном этапе производится обучение и сравнение пяти различных алгоритмов машинного обучения, применимых к задаче классификации писем на легитимные и фишинговые. Для каждого алгоритма выполняется подбор гиперпараметров с использованием `RandomizedSearchCV` и обучение на обучающей выборке.

Используемые алгоритмы:
- **Logistic Regression** — линейная модель с логистической функцией потерь
- **SVM (Linear Kernel)** — метод опорных векторов с линейным ядром
- **Random Forest** — ансамбль решающих деревьев
- **Naive Bayes** — байесовский классификатор
- **XGBoost** — градиентный бустинг на деревьях решений

Для оценки эффективности моделей используются метрики `accuracy`, `F1-мера`, `precision`, `recall`, а также замеряется время обучения каждой модели.


### 2.1 Logistic Regression


In [ ]:
# Logistic Regression
print("=" * 60)
print("Обучение Logistic Regression...")
print("=" * 60)

lr_model = LogisticRegression(random_state=42, max_iter=1000, n_jobs=-1)

# Параметры для RandomizedSearchCV
lr_param_dist = {
    'C': loguniform(1e-3, 1e2),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced']
}

lr_random_search = RandomizedSearchCV(
    estimator=lr_model,
    param_distributions=lr_param_dist,
    n_iter=20,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42,
    verbose=0  
)

print("Подбор гиперпараметров...")
with tqdm(total=20, desc="  Кандидаты", ncols=80) as pbar:
    start_time = time.time()
    lr_random_search.fit(X_train, y_train)
    lr_fit_time = time.time() - start_time
    pbar.update(20)

print(f"\n✓ Обучение завершено за {lr_fit_time:.2f} секунд")
print(f"Лучшие параметры: {lr_random_search.best_params_}")
print(f"Лучший F1-score (CV): {lr_random_search.best_score_:.4f}")

best_lr = lr_random_search.best_estimator_


### 2.2 SVM (Linear Kernel)


In [ ]:
# SVM (Linear Kernel)
print("=" * 60)
print("Обучение SVM (Linear Kernel)...")
print("=" * 60)

svm_model = SVC(kernel='linear', random_state=42, probability=True)

# Параметры для RandomizedSearchCV
svm_param_dist = {
    'C': loguniform(1e-3, 1e2),
    'max_iter': [1000, 2000, 3000],
    'class_weight': [None, 'balanced']
}

svm_random_search = RandomizedSearchCV(
    estimator=svm_model,
    param_distributions=svm_param_dist,
    n_iter=15,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42,
    verbose=0
)

print("Подбор гиперпараметров...")
with tqdm(total=15, desc="  Кандидаты", ncols=80) as pbar:
    start_time = time.time()
    svm_random_search.fit(X_train, y_train)
    svm_fit_time = time.time() - start_time
    pbar.update(15)

print(f"\n✓ Обучение завершено за {svm_fit_time:.2f} секунд")
print(f"Лучшие параметры: {svm_random_search.best_params_}")
print(f"Лучший F1-score (CV): {svm_random_search.best_score_:.4f}")

best_svm = svm_random_search.best_estimator_


### 2.3 Random Forest


In [ ]:
# Random Forest
print("=" * 60)
print("Обучение Random Forest...")
print("=" * 60)

rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)

# Параметры для RandomizedSearchCV
rf_param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': [10, 20, 30, None],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'class_weight': [None, 'balanced']
}

rf_random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=rf_param_dist,
    n_iter=20,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42,
    verbose=0
)

print("Подбор гиперпараметров...")
with tqdm(total=20, desc="  Кандидаты", ncols=80) as pbar:
    start_time = time.time()
    rf_random_search.fit(X_train, y_train)
    rf_fit_time = time.time() - start_time
    pbar.update(20)

print(f"\n✓ Обучение завершено за {rf_fit_time:.2f} секунд")
print(f"Лучшие параметры: {rf_random_search.best_params_}")
print(f"Лучший F1-score (CV): {rf_random_search.best_score_:.4f}")

best_rf = rf_random_search.best_estimator_


### 2.4 Naive Bayes


In [ ]:
# Naive Bayes
print("=" * 60)
print("Обучение Naive Bayes...")
print("=" * 60)

nb_model = MultinomialNB()

# Параметры для RandomizedSearchCV
nb_param_dist = {
    'alpha': uniform(0.1, 2.0)
}

nb_random_search = RandomizedSearchCV(
    estimator=nb_model,
    param_distributions=nb_param_dist,
    n_iter=15,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42,
    verbose=0
)

print("Подбор гиперпараметров...")
with tqdm(total=15, desc="  Кандидаты", ncols=80) as pbar:
    start_time = time.time()
    nb_random_search.fit(X_train, y_train)
    nb_fit_time = time.time() - start_time
    pbar.update(15)

print(f"\n✓ Обучение завершено за {nb_fit_time:.2f} секунд")
print(f"Лучшие параметры: {nb_random_search.best_params_}")
print(f"Лучший F1-score (CV): {nb_random_search.best_score_:.4f}")

best_nb = nb_random_search.best_estimator_


### 2.5 XGBoost


In [ ]:
# XGBoost
print("=" * 60)
print("Обучение XGBoost...")
print("=" * 60)

xgb_model = XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1)

# Параметры для RandomizedSearchCV
xgb_param_dist = {
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'n_estimators': randint(50, 300),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4)
}

xgb_random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_param_dist,
    n_iter=20,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42,
    verbose=0
)

print("Подбор гиперпараметров...")
with tqdm(total=20, desc="  Кандидаты", ncols=80) as pbar:
    start_time = time.time()
    xgb_random_search.fit(X_train, y_train)
    xgb_fit_time = time.time() - start_time
    pbar.update(20)

print(f"\n✓ Обучение завершено за {xgb_fit_time:.2f} секунд")
print(f"Лучшие параметры: {xgb_random_search.best_params_}")
print(f"Лучший F1-score (CV): {xgb_random_search.best_score_:.4f}")

best_xgb = xgb_random_search.best_estimator_


## 3. Кросс-валидация

Для каждой модели выполняется 5-кратная стратифицированная кросс-валидация на обучающей выборке. Это позволяет оценить стабильность и обобщающую способность моделей, а также получить усредненные метрики с оценкой стандартного отклонения для анализа вариативности результатов.


In [ ]:
# Словарь всех моделей
models_dict = {
    'Logistic Regression': best_lr,
    'SVM': best_svm,
    'Random Forest': best_rf,
    'Naive Bayes': best_nb,
    'XGBoost': best_xgb
}

# Метрики для кросс-валидации
scoring_metrics = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

# Выполнение кросс-валидации для всех моделей
cv_results_all = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Выполнение 5-кратной кросс-валидации...")
print("=" * 60)

for name, model in tqdm(models_dict.items(), desc="Модели", ncols=80):
    cv_results = cross_validate(
        model,
        X_train,
        y_train,
        cv=cv,
        scoring=scoring_metrics,
        return_train_score=False,
        n_jobs=-1
    )
    
    cv_results_all[name] = {
        'accuracy_mean': cv_results['test_accuracy'].mean(),
        'accuracy_std': cv_results['test_accuracy'].std(),
        'precision_mean': cv_results['test_precision'].mean(),
        'precision_std': cv_results['test_precision'].std(),
        'recall_mean': cv_results['test_recall'].mean(),
        'recall_std': cv_results['test_recall'].std(),
        'f1_mean': cv_results['test_f1'].mean(),
        'f1_std': cv_results['test_f1'].std()
    }
    
    print(f"\n{name}:")
    print(f"  Accuracy: {cv_results_all[name]['accuracy_mean']:.4f} ± {cv_results_all[name]['accuracy_std']:.4f}")
    print(f"  Precision: {cv_results_all[name]['precision_mean']:.4f} ± {cv_results_all[name]['precision_std']:.4f}")
    print(f"  Recall: {cv_results_all[name]['recall_mean']:.4f} ± {cv_results_all[name]['recall_std']:.4f}")
    print(f"  F1-score: {cv_results_all[name]['f1_mean']:.4f} ± {cv_results_all[name]['f1_std']:.4f}")

print("\n✓ Кросс-валидация завершена")


## 4. Оценка на валидационной выборке

Для каждой модели выполняется предсказание на валидационной выборке и расчет метрик качества: Accuracy, Precision, Recall, F1-score и ROC-AUC. Дополнительно измеряется время инференса для оценки производительности моделей.


In [ ]:
# Оценка на валидационной выборке
val_results = []

print("Оценка моделей на валидационной выборке...")
print("=" * 60)

for name, model in models_dict.items():
    # Предсказание
    start_time = time.time()
    y_pred = model.predict(X_val)
    inference_time = (time.time() - start_time) / len(X_val) * 1000  # в миллисекундах на образец
    
    # Вероятности для ROC-AUC
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_val)[:, 1]
    else:
        y_pred_proba = model.decision_function(X_val)
        # Нормализация для ROC-AUC
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        y_pred_proba = scaler.fit_transform(y_pred_proba.reshape(-1, 1)).ravel()
    
    # Метрики
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, zero_division=0)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    
    val_results.append({
        'Модель': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'ROC-AUC': roc_auc,
        'Inference time (ms)': inference_time
    })
    
    print(f"\n{name}:")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:   {recall:.4f}")
    print(f"  F1-score:  {f1:.4f}")
    print(f"  ROC-AUC:   {roc_auc:.4f}")
    print(f"  Inference: {inference_time:.2f} ms/образец")

val_results_df = pd.DataFrame(val_results)
print("\n" + "=" * 60)
print("Сводная таблица результатов на валидационной выборке:")
print("=" * 60)
display(val_results_df)


### 4.1 Матрицы ошибок на валидационной выборке

Матрицы ошибок позволяют визуально оценить качество классификации каждой модели и проанализировать распределение ошибок между классами.


In [ ]:
# Матрицы ошибок для всех моделей
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for idx, (name, model) in enumerate(models_dict.items()):
    y_pred = model.predict(X_val)
    cm = confusion_matrix(y_val, y_pred)
    
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        ax=axes[idx],
        cbar=False
    )
    axes[idx].set_title(f'{name}\nF1={f1_score(y_val, y_pred):.3f}')
    axes[idx].set_xlabel('Предсказано')
    axes[idx].set_ylabel('Реально')

plt.tight_layout()
plt.show()


## 5. Выбор лучшей модели

Выбор лучшей модели производится на основе следующих критериев:

1. **Максимальный F1-score** на валидационной выборке — баланс между точностью и полнотой
2. **Recall ≥ 0.95** — высокая полнота для обнаружения фишинговых писем (минимизация False Negatives)
3. **Inference time < 100ms** — приемлемая скорость работы для практического применения

Если нет моделей, удовлетворяющих всем критериям одновременно, выбирается модель с максимальным F1-score.


In [ ]:
# Выбор лучшей модели
# Фильтрация по критериям
candidates = val_results_df.copy()
candidates = candidates[
    (candidates['Recall'] >= 0.95) & 
    (candidates['Inference time (ms)'] < 100)
]

if len(candidates) == 0:
    # Если нет моделей, удовлетворяющих всем критериям, выбираем по F1-score
    print("⚠ Внимание: нет моделей, удовлетворяющих всем критериям (Recall ≥ 0.95 и Inference < 100ms)")
    print("Выбор лучшей модели по максимальному F1-score...")
    best_model_name = val_results_df.loc[val_results_df['F1-score'].idxmax(), 'Модель']
else:
    # Выбираем модель с максимальным F1-score среди кандидатов
    best_model_name = candidates.loc[candidates['F1-score'].idxmax(), 'Модель']

best_model = models_dict[best_model_name]
best_model_val_metrics = val_results_df[val_results_df['Модель'] == best_model_name].iloc[0]

print("=" * 60)
print(f"✓ Выбрана лучшая модель: {best_model_name}")
print("=" * 60)
print(f"Метрики на валидационной выборке:")
print(f"  Accuracy:  {best_model_val_metrics['Accuracy']:.4f}")
print(f"  Precision: {best_model_val_metrics['Precision']:.4f}")
print(f"  Recall:   {best_model_val_metrics['Recall']:.4f}")
print(f"  F1-score:  {best_model_val_metrics['F1-score']:.4f}")
print(f"  ROC-AUC:   {best_model_val_metrics['ROC-AUC']:.4f}")
print(f"  Inference: {best_model_val_metrics['Inference time (ms)']:.2f} ms/образец")


## 6. Калибровка весов агрегации

Для выбранной модели выполняется калибровка весов агрегации эвристического модуля (rules) и ML-модели. Используется grid search для поиска оптимальных весов `w_rules` и `w_ml`, где `w_ml = 1.0 - w_rules`.

**Формула агрегации:** `final_score = w_rules × risk_score + w_ml × confidence_score`

Где:
- `risk_score` — нормализованный риск-скор от эвристического модуля (0-1)
- `confidence_score` — уверенность ML-модели (0-1)
- `w_rules` — вес эвристического модуля
- `w_ml` — вес ML-модели

Для получения `risk_score` необходимо загрузить исходные данные писем и вычислить эвристические оценки с использованием модуля `rules_engine`.


In [ ]:
# Импорт модулей для вычисления risk_score
import sys
sys.path.append(str(BASE_DIR))

from src.email_parser import parse_email
from src.header_analyzer import analyze_headers
from src.rules_engine import evaluate_all_rules
from src.threat_intelligence import ThreatIntelligence

# Инициализация модуля Threat Intelligence
ti = ThreatIntelligence()

print("Модули для вычисления risk_score загружены")


In [ ]:
# Загрузка исходных данных для вычисления risk_score
# Предполагается, что в dataset_and_features.ipynb были сохранены исходные данные
# Если нет, нужно будет загрузить из email_dataset.csv

try:
    # Попытка загрузить исходные данные из CSV
    email_df = pd.read_csv(BASE_DIR / 'data' / 'processed' / 'email_dataset.csv')
    print(f"✓ Загружено {len(email_df)} записей из email_dataset.csv")
    
    # Получение индексов для валидационной выборки
    # Предполагается, что порядок сохранен при разделении
    # Если нет, нужно будет использовать другой способ сопоставления
    print("\n⚠ Внимание: для корректной калибровки весов необходимо")
    print("  обеспечить соответствие между X_val и исходными email данными.")
    print("  Если порядок не сохранен, потребуется дополнительная обработка.")
    
except FileNotFoundError:
    print("⚠ Файл email_dataset.csv не найден.")
    print("  Для калибровки весов необходимо загрузить исходные данные писем.")
    print("  Временно используем синтетические risk_score для демонстрации.")
    
    # Создание синтетических risk_score для демонстрации
    # В реальном сценарии нужно вычислять из исходных данных
    np.random.seed(42)
    # Генерируем risk_score на основе предсказаний модели (для демонстрации)
    # В реальности это должно быть из evaluate_all_rules()
    synthetic_risk_scores = np.random.randint(0, 100, size=len(y_val))
    email_df = None


In [ ]:
# Функция для вычисления risk_score для одного письма
def compute_risk_score(email_content: str) -> int:
    """
    Вычисление risk_score для письма с использованием эвристических правил
    
    Args:
        email_content: исходное содержимое письма (EML формат)
        
    Returns:
        int: risk_score (0-100)
    """
    try:
        # Парсинг письма
        parsed = parse_email(email_content)
        
        # Анализ заголовков
        headers = {
            'from': parsed.get('from', ''),
            'to': parsed.get('to', ''),
            'subject': parsed.get('subject', ''),
            'auth_results': parsed.get('auth_results', {}),
            'reply_to': parsed.get('reply_to', ''),
            'return_path': parsed.get('return_path', ''),
            'received_headers': parsed.get('received_headers', []),
            'references': parsed.get('references', '')
        }
        header_analysis = analyze_headers(headers)
        
        # Threat Intelligence проверки
        domains = parsed.get('domains', [])
        ips = parsed.get('ips', [])
        ti_results = {}
        
        # Проверка доменов (ограничиваем для скорости)
        for domain in domains[:5]:
            ti_results[domain] = ti.check_domain_reputation(domain)
        
        # Проверка IP (ограничиваем для скорости)
        for ip in ips[:5]:
            ti_results[ip] = ti.check_ip_reputation(ip)
        
        # Оценка всех правил
        rules_result = evaluate_all_rules(header_analysis, parsed, ti_results)
        
        return rules_result['risk_score']
    
    except Exception as e:
        print(f"Ошибка при вычислении risk_score: {e}")
        return 0

print("Функция compute_risk_score определена")


In [ ]:
# Вычисление risk_score для валидационной выборки
# ВАЖНО: Это может занять много времени, если данных много
# Для ускорения можно использовать кэширование или предвычисленные значения

print("Вычисление risk_score для валидационной выборки...")
print("⚠ Это может занять значительное время...")

# Получение confidence scores от ML модели
y_val_confidence = best_model.predict_proba(X_val)[:, 1] if hasattr(best_model, 'predict_proba') else None

if y_val_confidence is None:
    # Если нет predict_proba, используем decision_function
    decision_scores = best_model.decision_function(X_val)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    y_val_confidence = scaler.fit_transform(decision_scores.reshape(-1, 1)).ravel()

# Вычисление risk_score
# Если есть исходные данные, вычисляем реальные risk_score
# Иначе используем синтетические для демонстрации

if email_df is not None and 'email_content' in email_df.columns:
    # Реальные вычисления (может быть медленно)
    print("Вычисление реальных risk_score из исходных данных...")
    risk_scores_val = []
    for idx in tqdm(range(len(X_val)), desc="  Обработка писем", ncols=80):
        # Здесь нужно правильно сопоставить индексы
        # Это упрощенный пример - в реальности нужна правильная индексация
        if idx < len(email_df):
            risk_score = compute_risk_score(email_df.iloc[idx]['email_content'])
            risk_scores_val.append(risk_score)
        else:
            risk_scores_val.append(0)
    risk_scores_val = np.array(risk_scores_val)
else:
    # Синтетические risk_score для демонстрации
    print("Использование синтетических risk_score для демонстрации...")
    print("⚠ В реальном сценарии необходимо загрузить исходные данные писем")
    np.random.seed(42)
    # Генерируем risk_score с некоторой корреляцией с метками
    risk_scores_val = np.random.randint(0, 100, size=len(y_val))
    # Увеличиваем risk_score для фишинговых писем
    risk_scores_val[y_val == 1] = np.random.randint(30, 100, size=np.sum(y_val == 1))

# Нормализация risk_score к диапазону [0, 1]
risk_scores_val_normalized = risk_scores_val / 100.0

print(f"\n✓ Вычислено {len(risk_scores_val)} risk_score")
print(f"  Средний risk_score: {risk_scores_val.mean():.2f}")
print(f"  Медианный risk_score: {np.median(risk_scores_val):.2f}")


In [ ]:
# Grid search для калибровки весов
print("=" * 60)
print("Калибровка весов агрегации...")
print("=" * 60)

# Диапазон весов для rules: от 0.0 до 0.7 с шагом 0.1
w_rules_candidates = np.arange(0.0, 0.8, 0.1)
w_ml_candidates = 1.0 - w_rules_candidates

best_f1 = 0
best_w_rules = 0.3
best_w_ml = 0.7
best_final_scores = None
best_y_pred_aggregated = None

results_calibration = []

print("Поиск оптимальных весов...")
for w_rules, w_ml in tqdm(zip(w_rules_candidates, w_ml_candidates), 
                          total=len(w_rules_candidates), 
                          desc="  Комбинации весов", 
                          ncols=80):
    # Агрегация скоров
    final_scores = w_rules * risk_scores_val_normalized + w_ml * y_val_confidence
    
    # Поиск оптимального порога
    thresholds = np.arange(0.1, 1.0, 0.05)
    best_threshold = 0.5
    best_f1_threshold = 0
    
    for threshold in thresholds:
        y_pred_thresh = (final_scores >= threshold).astype(int)
        f1_thresh = f1_score(y_val, y_pred_thresh)
        
        if f1_thresh > best_f1_threshold:
            best_f1_threshold = f1_thresh
            best_threshold = threshold
    
    # Финальные предсказания с оптимальным порогом
    y_pred_agg = (final_scores >= best_threshold).astype(int)
    
    # Метрики
    accuracy = accuracy_score(y_val, y_pred_agg)
    precision = precision_score(y_val, y_pred_agg, zero_division=0)
    recall = recall_score(y_val, y_pred_agg)
    f1 = f1_score(y_val, y_pred_agg)
    
    results_calibration.append({
        'w_rules': w_rules,
        'w_ml': w_ml,
        'threshold': best_threshold,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1
    })
    
    if f1 > best_f1:
        best_f1 = f1
        best_w_rules = w_rules
        best_w_ml = w_ml
        best_final_scores = final_scores.copy()
        best_y_pred_aggregated = y_pred_agg.copy()
        best_threshold_final = best_threshold

calibration_results_df = pd.DataFrame(results_calibration)

print("\n" + "=" * 60)
print("Результаты калибровки весов:")
print("=" * 60)
display(calibration_results_df.sort_values('F1-score', ascending=False))

print(f"\n✓ Оптимальные веса:")
print(f"  w_rules = {best_w_rules:.1f}")
print(f"  w_ml = {best_w_ml:.1f}")
print(f"  threshold = {best_threshold_final:.2f}")
print(f"  F1-score (val) = {best_f1:.4f}")


### 6.1 Визуализация результатов калибровки

Визуализация результатов калибровки весов позволяет оценить влияние различных комбинаций весов на метрики качества и выбрать оптимальные значения.


In [ ]:
# Визуализация результатов калибровки
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# График F1-score в зависимости от w_rules
axes[0].plot(calibration_results_df['w_rules'], calibration_results_df['F1-score'], 
             marker='o', linewidth=2, markersize=8)
axes[0].axvline(x=best_w_rules, color='r', linestyle='--', label=f'Оптимум (w_rules={best_w_rules:.1f})')
axes[0].set_xlabel('Вес правил (w_rules)')
axes[0].set_ylabel('F1-score')
axes[0].set_title('Зависимость F1-score от веса правил')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Тепловая карта метрик
metrics_pivot = calibration_results_df.pivot_table(
    values=['F1-score', 'Precision', 'Recall'],
    index='w_rules'
)

sns.heatmap(
    metrics_pivot,
    annot=True,
    fmt='.3f',
    cmap='YlOrRd',
    ax=axes[1],
    cbar_kws={'label': 'Значение метрики'}
)
axes[1].set_title('Метрики в зависимости от веса правил')
axes[1].set_xlabel('Метрика')
axes[1].set_ylabel('Вес правил (w_rules)')

plt.tight_layout()
plt.show()


## 7. Оценка на тестовой выборке

Финальная оценка выбранной модели с оптимальными весами агрегации на тестовой выборке. На данном этапе производится вычисление метрик качества, анализ матрицы ошибок и сравнение результатов модели только с ML-компонентом и модели с агрегацией ML и эвристических правил.


In [ ]:
# Вычисление risk_score для тестовой выборки
print("Вычисление risk_score для тестовой выборки...")

if email_df is not None and 'email_content' in email_df.columns:
    # Реальные вычисления
    risk_scores_test = []
    for idx in tqdm(range(len(X_test)), desc="  Обработка писем", ncols=80):
        if idx < len(email_df):
            risk_score = compute_risk_score(email_df.iloc[idx]['email_content'])
            risk_scores_test.append(risk_score)
        else:
            risk_scores_test.append(0)
    risk_scores_test = np.array(risk_scores_test)
else:
    # Синтетические risk_score для демонстрации
    np.random.seed(42)
    risk_scores_test = np.random.randint(0, 100, size=len(y_test))
    risk_scores_test[y_test == 1] = np.random.randint(30, 100, size=np.sum(y_test == 1))

# Нормализация
risk_scores_test_normalized = risk_scores_test / 100.0

# Confidence scores от ML модели
y_test_confidence = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model, 'predict_proba') else None

if y_test_confidence is None:
    decision_scores = best_model.decision_function(X_test)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    y_test_confidence = scaler.fit_transform(decision_scores.reshape(-1, 1)).ravel()

# Агрегация с оптимальными весами
final_scores_test = best_w_rules * risk_scores_test_normalized + best_w_ml * y_test_confidence

# Предсказания с оптимальным порогом
y_pred_test = (final_scores_test >= best_threshold_final).astype(int)

# Метрики на тестовой выборке
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, zero_division=0)
test_recall = recall_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)
test_roc_auc = roc_auc_score(y_test, final_scores_test)

# Матрица ошибок
test_cm = confusion_matrix(y_test, y_pred_test)

print("=" * 60)
print("Результаты на тестовой выборке:")
print("=" * 60)
print(f"Accuracy:  {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:   {test_recall:.4f}")
print(f"F1-score:  {test_f1:.4f}")
print(f"ROC-AUC:   {test_roc_auc:.4f}")

print("\nМатрица ошибок:")
print(test_cm)

# Анализ False Positive и False Negative
tn, fp, fn, tp = test_cm.ravel()
print(f"\nАнализ ошибок:")
print(f"  True Negatives (TN):  {tn}")
print(f"  False Positives (FP): {fp} ({fp/len(y_test)*100:.2f}%)")
print(f"  False Negatives (FN): {fn} ({fn/len(y_test)*100:.2f}%)")
print(f"  True Positives (TP):  {tp}")

test_metrics = {
    'accuracy': float(test_accuracy),
    'precision': float(test_precision),
    'recall': float(test_recall),
    'f1_score': float(test_f1),
    'roc_auc': float(test_roc_auc),
    'confusion_matrix': {
        'tn': int(tn),
        'fp': int(fp),
        'fn': int(fn),
        'tp': int(tp)
    }
}


### 7.1 Визуализация результатов на тестовой выборке

Визуализация включает матрицу ошибок на тестовой выборке и сравнительный анализ метрик модели только с ML-компонентом и модели с агрегацией ML и эвристических правил.


In [ ]:
# Визуализация результатов
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Матрица ошибок
sns.heatmap(
    test_cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    ax=axes[0],
    cbar_kws={'label': 'Количество'}
)
axes[0].set_title(f'Матрица ошибок на тестовой выборке\n({best_model_name} с агрегацией)')
axes[0].set_xlabel('Предсказано')
axes[0].set_ylabel('Реально')
axes[0].set_xticklabels(['Legitimate', 'Phishing'])
axes[0].set_yticklabels(['Legitimate', 'Phishing'])

# Сравнение метрик: только ML vs ML + Rules
metrics_comparison = {
    'Метрика': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
    'Только ML': [
        accuracy_score(y_test, best_model.predict(X_test)),
        precision_score(y_test, best_model.predict(X_test), zero_division=0),
        recall_score(y_test, best_model.predict(X_test)),
        f1_score(y_test, best_model.predict(X_test))
    ],
    'ML + Rules': [test_accuracy, test_precision, test_recall, test_f1]
}

comparison_df = pd.DataFrame(metrics_comparison)
comparison_melted = comparison_df.melt(
    id_vars='Метрика',
    value_vars=['Только ML', 'ML + Rules'],
    var_name='Модель',
    value_name='Значение'
)

sns.barplot(
    data=comparison_melted,
    x='Метрика',
    y='Значение',
    hue='Модель',
    palette='Set2',
    ax=axes[1]
)
axes[1].set_title('Сравнение метрик: только ML vs ML + Rules')
axes[1].set_ylabel('Значение метрики')
axes[1].set_ylim(0, 1)
axes[1].grid(axis='y', alpha=0.3)
axes[1].legend(title='Модель')

plt.tight_layout()
plt.show()


## 8. Сравнительная таблица метрик

Сводная таблица метрик всех моделей на валидационной выборке и финальной модели с агрегацией на тестовой выборке. Таблица сохраняется в формате CSV для использования в ВКР.


In [ ]:
# Создание сравнительной таблицы для ВКР
comparison_table = []

# Добавляем результаты всех моделей на валидационной выборке
for _, row in val_results_df.iterrows():
    comparison_table.append({
        'Модель': row['Модель'],
        'Выборка': 'Validation',
        'Accuracy': row['Accuracy'],
        'Precision': row['Precision'],
        'Recall': row['Recall'],
        'F1-score': row['F1-score'],
        'ROC-AUC': row['ROC-AUC']
    })

# Добавляем результаты лучшей модели с агрегацией на тестовой выборке
comparison_table.append({
    'Модель': f'{best_model_name} + Rules (aggregated)',
    'Выборка': 'Test',
    'Accuracy': test_accuracy,
    'Precision': test_precision,
    'Recall': test_recall,
    'F1-score': test_f1,
    'ROC-AUC': test_roc_auc
})

comparison_table_df = pd.DataFrame(comparison_table)

print("=" * 80)
print("Сравнительная таблица метрик (для ВКР):")
print("=" * 80)
display(comparison_table_df.round(4))

# Сохранение таблицы в CSV
comparison_table_df.to_csv(MODELS_DIR / 'comparison_metrics.csv', index=False, encoding='utf-8-sig')
print(f"\n✓ Таблица сохранена: {MODELS_DIR / 'comparison_metrics.csv'}")


In [ ]:
# Сохранение лучшей модели
print("Сохранение лучшей модели...")
with open(MODELS_DIR / 'best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
print(f"✓ Модель сохранена: {MODELS_DIR / 'best_model.pkl'}")

# Сохранение оптимальных весов агрегации
aggregator_weights = {
    'rules_weight': float(best_w_rules),
    'ml_weight': float(best_w_ml),
    'optimal_threshold': float(best_threshold_final),
    'val_f1_score': float(best_f1),
    'model_name': best_model_name
}

print("\nСохранение весов агрегации...")
with open(MODELS_DIR / 'aggregator_weights.json', 'w', encoding='utf-8') as f:
    json.dump(aggregator_weights, f, ensure_ascii=False, indent=2)
print(f"✓ Веса сохранены: {MODELS_DIR / 'aggregator_weights.json'}")

# Сохранение метрик на тестовой выборке
print("\nСохранение метрик на тестовой выборке...")
with open(MODELS_DIR / 'test_metrics.json', 'w', encoding='utf-8') as f:
    json.dump(test_metrics, f, ensure_ascii=False, indent=2)
print(f"✓ Метрики сохранены: {MODELS_DIR / 'test_metrics.json'}")

print("\n✓ Все результаты успешно сохранены!")
